In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [24]:
import rasterio
import rasterio.transform
import fiona
import shapely
import shapely.geometry
import pyproj

from matplotlib import pyplot as plt
import numpy as np
import mpl_toolkits.basemap as pbm

In [6]:
# warping crs
def warp_xy(x, y, old_crs, new_crs):
    """Warps a set of points from old_crs to new_crs."""
    if old_crs == new_crs:
        return x,y

    old_crs_proj = pyproj.Proj(old_crs)
    new_crs_proj = pyproj.Proj(new_crs)
    return pyproj.transform(old_crs_proj, new_crs_proj, x,y)

def warp_shapely(shp, old_crs, new_crs):
    """Uses proj to reproject shapes, NOT IN PLACE"""
    if old_crs['init'] == new_crs['init']:
        return shp

    old_crs_proj = pyproj.Proj(old_crs)
    new_crs_proj = pyproj.Proj(new_crs)
    return shapely.ops.transform(lambda x,y:pyproj.transform(old_crs_proj, new_crs_proj, x,y), shp)

def warp_shape(feature, old_crs, new_crs):
    """Uses proj to reproject shapes, IN PLACE"""
    if old_crs == new_crs:
        return
    if len(feature['geometry']['coordinates']) is 0:
        return

    # find the dimension -- can't trust the shape
    dim = -1
    ptr = feature['geometry']['coordinates']
    done = False
    while not done:
        if hasattr(ptr, '__len__'):        
            assert(len(ptr) is not 0)
            dim += 1
            ptr = ptr[0]
        else:
            done = True

    if dim == 0:
        # point
        x,y = warp_xy(np.array([feature['geometry']['coordinates'][0],]), np.array([feature['geometry']['coordinates'][1],]), old_crs, new_crs)
        feature['geometry']['coordinates'][0] = x[0]
        feature['geometry']['coordinates'][1] = x[1]
    elif dim == 1:
        # line-like or polygon with no holes
        coords = np.array(feature['geometry']['coordinates'],'d')
        assert(len(coords.shape) is 2 and coords.shape[1] in [2,3] )
        x,y = warp_xy(coords[:,0], coords[:,1], old_crs, new_crs)
        new_coords = [xy for xy in zip(x,y)]
        feature['geometry']['coordinates'] = new_coords
    elif dim == 2:
        # multi-line or polygon with holes
        for i in range(len(feature['geometry']['coordinates'])):
            coords = np.array(feature['geometry']['coordinates'][i],'d')
            assert(len(coords.shape) is 2 and coords.shape[1] in [2,3])
            x,y = warp_xy(coords[:,0], coords[:,1], old_crs, new_crs)
            new_coords = [xy for xy in zip(x,y)]
            feature['geometry']['coordinates'][i] = new_coords
    elif dim == 3:
        # multi-polygon
        for i in range(len(feature['geometry']['coordinates'])):
            for j in range(len(feature['geometry']['coordinates'][i])):
                coords = np.array(feature['geometry']['coordinates'][i][j],'d')
                assert(len(coords.shape) is 2 and coords.shape[1] in [2,3])
                x,y = warp_xy(coords[:,0], coords[:,1], old_crs, new_crs)
                new_coords = [xy for xy in zip(x,y)]
                feature['geometry']['coordinates'][i][j] = new_coords
            
                    
    


In [3]:
# import MSA shapes
with fiona.open('./data/MSAs/tl_2017_us_cbsa/tl_2017_us_cbsa.shp', 'r') as fid:
    msas = list(fid)
    msas_profile = fid.profile
    
print("we found {} shapes".format(len(msas)))

we found 945 shapes


In [26]:
# plot the shapes
plt.figure()

bm = pbm.Basemap(llcrnrlon=-130, llcrnrlat=23, urcrnrlon=-68, urcrnrlat=52, resolution='h', epsg=msas_profile['crs']['init'][5:])
bm.drawcountries(linewidth=1, color='darkgoldenrod')
bm.drawstates(color='darkgoldenrod')

# -- convert to shapely
for shp in msas:
    shply = shapely.geometry.shape(shp['geometry'])
    if type(shply) is shapely.geometry.Polygon:
        xy = np.array(shply.exterior.xy)
        plt.plot(xy[0], xy[1], 'b')
    elif type(shply) is shapely.geometry.MultiPolygon:
        for poly in shply:
            xy = np.array(poly.exterior.xy)
            plt.plot(xy[0], xy[1], 'r')
    else:
        print('found an object of type: {}'.format(type(shply)))
        

plt.show()



/Users/uec/codes/anaconda/3/envs/ats_meshing_20190613/lib/python3.7/site-packages/ipykernel_launcher.py:4: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  after removing the cwd from sys.path.
/Users/uec/codes/anaconda/3/envs/ats_meshing_20190613/lib/python3.7/site-packages/ipykernel_launcher.py:5: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  """
/Users/uec/codes/anaconda/3/envs/ats_meshing_20190613/lib/python3.7/site-packages/ipykernel_launcher.py:6: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  


In [4]:
with rasterio.open('./data/NLCD/NLCD_2016_Impervious_L48_20190405.img','r') as fid:
    rprof = fid.profile


In [5]:
print('What CRS are we working in?')
print('  crs of MSAs:', msas_profile['crs'])
print('  crs of NLCD:', rprof['crs'])


What CRS are we working in?
  crs of MSAs: {'init': 'epsg:4269'}
  crs of NLCD: PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],TOWGS84[0,0,0,-0,-0,-0,0],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1]]


In [9]:
# changed coordinate system from the native crs of the shapefile to the native crs of the raster
for shp in msas:
    warp_shape(shp, msas_profile['crs'], rprof['crs'])

In [25]:
# exercise: figure out the pixel coordinate box that covers a given MSA
msa = msas[100]
msa_shply = shapely.geometry.shape(msa['geometry'])
msa_bounds = msa_shply.bounds

transform = rprof['transform']

# replace me start
centroid = msa_shply.centroid.xy
x,y = centroid[0], centroid[1]
i,j = rasterio.transform.rowcol(transform, x, y)

print('MSA is centered at pixel {}, {}'.format(i,j))

#with rasterio.open('./data/NLCD/NLCD_2016_Impervious_L48_20190405.img','r') as fid:
#    msa_boxed_imp_surf = fid.read(1, window=Window(0, 0, 512, 256))



plt.imshow(msa_boxed_imp_surf, extent=[msa_bounds[0], msa_bounds[2], msa_bounds[1], msa_bounds[3]])
msa_shply_xy = np.array(msa_shply.exterior.xy)
plt.plot(msa_shply_xy[0], msa_shply_xy[1])
plt.show()

True
MSA is centered at pixel [43273], [130738]
